In [98]:
from langchain.llms.base import LLM
from typing import Optional, List
import requests

class ClarinChatLLM(LLM):
    model_id: str
    api_key: str
    temperature: float = 0.0

    @property
    def _llm_type(self) -> str:
        return "clarin-chat"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        url = "https://services.clarin-pl.eu/api/v1/oapi/chat/completions"
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json"
        }
        messages = [{"role": "user", "content": prompt}]
        data = {
            "model": self.model_id,
            "messages": messages,
            "temperature": self.temperature
        }
        response = requests.post(url, json=data, headers=headers)
        if response.status_code == 200:
            try:
                json_data = response.json()
                return json_data["choices"][0]["message"]["content"]
            except Exception as e:
                print(f"Invalid response format: {e}")
                return None
        else:
            print(f"Request failed with status {response.status_code}: {response.text}")
            return None


In [2]:
with open("key.txt", "r") as file:
    API_KEY_CLARIN = file.read().strip()

In [99]:
clarin_llm = ClarinChatLLM(
    model_id="bielik",
    api_key=API_KEY_CLARIN
)

In [65]:
def parse_characters(file_path):
    characters = {}
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    
    current_character = None
    for line in lines:
        line = line.strip()
        if line.startswith("Osoba:"):
            current_character = line.split(":")[1].strip()
            characters[current_character] = {"gender": None, "character": None}
        elif line.startswith("płeć:") and current_character:
            characters[current_character]["gender"] = line.split(":")[1].strip()
        elif line.startswith("charakter:") and current_character:
            characters[current_character]["character"] = line.split(":")[1].strip()
    
    return characters

characters_dict = parse_characters("example_context.txt")

In [66]:
characters_dict

{'Geralt': {'gender': 'mężczyzna', 'character': None},
 'Yennefer': {'gender': 'kobieta', 'character': None},
 'Troll': {'gender': 'nijaka',
  'character': 'wypowiada się w formie bezosobowej, nie umie odmieniać przez płeć, wypowiada się na poziomie dziecka, mowi "ja być" zamiast "ja jestem", albo "ty być" zamiast "ty jesteś"'}}

In [ ]:
from langchain.prompts import PromptTemplate

translation_prompt = PromptTemplate(
    input_variables=["speaker", "listener", "text", "speaker_gender", "listener_gender", "speaker_character", "listener_character"],
    template="""
Twoim zadaniem jest przetłumaczyć wypowiedź z angielskiego na polski, stylizując ją zgodnie z osobowością, płcią i sposobem mówienia postaci.

Mówiący: {speaker}
Płeć mówiącego: {speaker_gender}
Sposób mówienia: {speaker_character}

Słuchacz: {listener}
Płeć słuchacza: {listener_gender}

Zachowaj styl wypowiedzi mówiącego. Jeśli postać mówi w specyficzny sposób, odwzoruj to w tłumaczeniu.

Tekst do przetłumaczenia: "{text}"

Przetłumaczony tekst:
"""
)

In [80]:
from langchain.chains import LLMChain
translator_chain = LLMChain(
    llm=clarin_llm,  # your wrapped CLARIN LLM
    prompt=translation_prompt
)

In [81]:
def translate_with_context(speaker, listener, text):
    # Pobieramy informacje o mówiącym i słuchaczu
    speaker_info = characters_dict.get(speaker, {"gender": "nieznana", "character": "nieznany"})
    listener_info = characters_dict.get(listener, {"gender": "nieznana"})
    
    speaker_gender = speaker_info["gender"]
    listener_gender = listener_info["gender"]
    speaker_character = speaker_info["character"]
    
    return translator_chain.run({
        "speaker": speaker,
        "listener": listener,
        "text": text,
        "speaker_gender": speaker_gender,
        "listener_gender": listener_gender,
        "speaker_character": speaker_character,
    })

In [82]:
print(translate_with_context(
    speaker="Troll",
    listener="Yennefer",
    text="I am the best artist, are you a good artist too?"
))


Przetłumaczony tekst zgodnie ze stylem i charakterystyką wskazanych postaci:

"Ja być najlepszym artystą, ty też być dobrym artystą?"

Wypowiedź ta odzwierciedla prosty, dziecinny sposób mówienia trolla, który używa formy bezosobowej i ma trudności z odmianą przez płeć. Jednocześnie zachowuje ona sens oryginalnego pytania o umiejętności artystyczne rozmówcy.


In [83]:
print(translate_with_context(
    speaker="Yennefer",
    listener="Troll",
    text="I am the best artist, are you a good artist too?"
))

Oto przetłumaczona wypowiedź stylizowana na charakterystyczny sposób mówienia Yennefer, z uwzględnieniem jej osobowości jako kobiety:

"Jestem najlepszą artystką, czy ty też jesteś dobrym artystą?"

Yennefer często używa lekko wyniosłego tonu, podkreślając swoją wyższość i pewność siebie. W tym zdaniu można wyczuć nutę wyzwania lub prowokacji, sugerując, że rozmówca powinien udowodnić swoje umiejętności, aby dorównać jej poziomowi. Jednocześnie pytanie o bycie "dobrym artystą" może być interpretowane jako subtelne umniejszenie wartości rozmówcy, co jest typowe dla jej sposobu komunikacji.


In [84]:
print(translate_with_context(
    speaker="Geralt",
    listener="Troll",
    text="I am the best artist, are you a good artist too?"
))

Geralt (mężczyzna):
"Jestem najlepszym artystą w swojej dziedzinie, czy ty też jesteś dobrym artystą?"

Troll (nijaki):
"Jam jest mistrzem w swoim fachu, czy i ty posiadasz umiejętności godne pochwały?"


In [85]:
print(translate_with_context(
    speaker="Geralt",
    listener="Yennefer",
    text="I am the best artist, are you a good artist too?"
))

Geralt (mężczyzna):
"No cóż, ja jestem najlepszym Wiedźminem, jakiego znam... A ty? Czy jesteś dobrą czarodziejką?"

Yennefer (kobieta):
"Och, Geralt, nie bądź taki skromny! Wiesz doskonale, że jesteś legendarnym Wiedźminem. A co do mnie - powiedzmy, że potrafię rzucić kilka zaklęć lepiej niż inne."


In [114]:
translation_step1 = PromptTemplate(
    input_variables=["sentence"],
    template="""
Przetłumacz na polski i nie dodawaj żadnych dodatkowych wyjaśnień, napisz tylko samo tłumaczenie: {sentence}
"""
)

translation_step2 = PromptTemplate(
    input_variables=["speaker", "listener", "translated_sentence", "speaker_gender", "listener_gender", "speaker_character"],
    template="""

zdanie wypowiadane jest przez {speaker_gender} do {listener_gender}

Przepisz podane zdanie na polski zgodnie z podanym kotekstem i nie dodawaj żadnych dodatkowych wyjaśnień, napisz tylko samo tłumaczenie: "{translated_sentence}"

"""
)

In [115]:
step1_chain = LLMChain(llm=clarin_llm, prompt=translation_step1)
step2_chain = LLMChain(llm=clarin_llm, prompt=translation_step2)

def translate_with_two_steps(speaker, listener, sentence):

    speaker_info = characters_dict.get(speaker, {"gender": "nieznana", "character": "nieznany"})
    listener_info = characters_dict.get(listener, {"gender": "nieznana"})
    
    speaker_gender = speaker_info["gender"]
    listener_gender = listener_info["gender"]
    speaker_character = speaker_info["character"]

    step1_result = step1_chain.run({
        "sentence": sentence
    })
    print(step1_result)
    step2_result = step2_chain.run({
        "translated_sentence": step1_result,
        "speaker": speaker,
        "listener": listener,
        "speaker_gender": speaker_gender,
        "listener_gender": listener_gender,
        "speaker_character": speaker_character,
    })
    
    return step2_result

In [116]:
print(translate_with_two_steps(
    speaker="Geralt",
    listener="Yennefer",
    sentence="I am the best artist, are you a good artist too?"
))

Jestem najlepszym artystą, czy ty też jesteś dobrym artystą?
"Jestem najlepszym artystą, czy ty też jesteś dobrym artystą?"
